See KNN Impute Work folder for specific work through for each model.  This file contains the cleaned version to impute and clean quickly.

# Imports

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import pickle
from matplotlib.cbook import boxplot_stats 

# Variables

In [2]:
url = "https://raw.githubusercontent.com/joepollastrini/Loan-Prediction/master"
train_name = 'train_loan_data.csv'
test_name = 'test_loan_data.csv'
    
direct = os.getcwd()

loan_term_model_name = 'knn_Loan_Term.sav'
loan_term_cols = ['IncomePerMember', 'FamilyIncome']

loan_amount_model_name = 'knn_Loan_Amount.sav'
loan_amount_cols = ['FamilyIncome', 'LTG_<15', 'LTG_15', 'LTG_1530', 'LTG_30']

self_employed_model_name = 'knn_Self_Employed.sav'
self_employed_cols = ['FamilyIncome']

credit_history_model_name = 'knn_Credit_History.sav'
credit_history_cols = ['LTG_<15', 'LTG_15', 'LTG_1530', 'LTG_30', 'LTG_>30']

# Functions

In [3]:
def data_grab_online(base, name, out='output.csv'):
    #get data from url
    u = base + '/' + name
    r = requests.get(u).content
    df = pd.read_csv(u)
    return df

In [4]:
def gender_impute_and_dummy(row):
    #if unmarried and 1 dependent female, else male
    if pd.isnull(row['Gender']):
        if row['Married'] == 'No' and row['Dependents'] == '1':
            return 0
        else:
            return 1
    else:
        if row['Gender'] == 'Male':
            return 1
        else:
            return 0

In [5]:
def married_impute_and_dummy(x):
    #if female, not married, otherwise married
    if pd.isnull(x['Married']):
        if x['Gender'] == 'Female':
            return 0
        else:
            return 1
    else:
        if x['Married'] == 'Yes':
            return 1
        else:
            return 0

In [6]:
def dependents_impute_and_ordinal(x):
    #if married, one dependent, otherwise none
    #convert dependents to ordinal int as well
    if pd.isnull(x['Dependents']):
        if x['Married_IO'] == 1:
            return 1
        else:
            return 0
    elif x['Dependents'] == '3+':
        return 3
    else:
        return int(x['Dependents'])

In [7]:
def dual_income_dummy(x):
    if x['Married'] == 'Yes':
        if x['CoapplicantIncome'] > 0:
            return 1
        else:
            return 0
    else:
        return 0

In [8]:
def clean(df):
    col_drop = []
    col_rename = {}
    
    # GENDER #
    df['Male_IO'] = df.apply(lambda x: gender_impute_and_dummy(x), axis=1)
    col_drop.append('Gender')
    
    # MARRIED #
    df['Married_IO'] = df.apply(lambda x: married_impute_and_dummy(x), axis=1)
    col_drop.append('Married')
    
    # DEPENDENTS #
    df['Dependents2'] = df.apply(lambda x: dependents_impute_and_ordinal(x), axis=1)
    col_drop.append('Dependents')
    col_rename['Dependents2'] = 'Dependents'
    
    # FAMILY SIZE #
    df['FamilySize'] = df['Dependents2'] + df['Married_IO'] + 1
    
    # EDUCATION #
    df['Education_IO'] = df['Education'].apply(lambda x: 1 if x == 'Graduate' else 0)
    col_drop.append('Education')

    # INCOME #
    df['FamilyIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']
    df['DualIncome_IO'] = df.apply(lambda x: dual_income_dummy(x), axis=1)
    col_drop.extend(['ApplicantIncome', 'CoapplicantIncome'])
    
    # PROPERTY AREA #
    df['PA_Urban'] = df['Property_Area'].apply(lambda x: 1 if x == 'Urban' else 0)
    df['PA_Rural'] = df['Property_Area'].apply(lambda x: 1 if x == 'Rural' else 0)
    df['PA_Semiurban'] = df['Property_Area'].apply(lambda x: 1 if x == 'Semiurban' else 0)
    col_drop.append('Property_Area')
    
    # INCOME PER FAMILY MEMBER #
    df['IncomePerMember'] = df['FamilyIncome'] / df['FamilySize']
    
    # LOAN STATUS #
    try:
        df['Loan_Status'].replace('Y', 1, inplace=True)
        df['Loan_Status'].replace('N', 0, inplace=True)
    except KeyError:
        pass
    
    #column cleaning
    df.drop(columns = col_drop, inplace=True)
    df.rename(columns=col_rename, inplace=True)
    return df

In [9]:
def loan_group(x):
    if np.isnan(x):
        return x
    else:
        x = int(x)
        if x == 360:
            return '30'
        elif x == 180:
            return '15'
        elif x < 180:
            return '<15'
        elif x > 180 and x < 360:
            return '(15, 30)'
        elif x > 360:
            return '>30'
        else:
            return x

In [10]:
def predict_func(model, row, x_cols):
    valueList = []
    for col in x_cols:
        value = row[col]
        valueList.append(value)

    df = pd.DataFrame([valueList])
    
    prediction = model.predict(df)[0]
    return prediction

In [11]:
def new_loan_term(x):
    if '<' in x:
        return 7.5*12.0
    elif '(' in x:
        return 22.5*12.0
    elif '>' in x:
        return 480.0
    else:
        return float(x)*12.0

In [12]:
def clean2(df, model, cols):
    #group loan terms, don't change nan values
    df['LoanTermGroups'] = df['Loan_Amount_Term'].apply(lambda x: loan_group(x))
    
    #create predictions using model
    df['LTA_Predict'] = df.apply(lambda x: predict_func(model, x, cols), axis=1)
    
    #replace missing values from loan term groups with predicted values
    df['LoanTermGroups'].fillna(df['LTA_Predict'], inplace=True)
    
    #drop columns
    df.drop(columns = ['Loan_Amount_Term', 'LTA_Predict'], inplace=True)
    
    #create loan amount term monthly for calculations
    df['Loan_Amount_Term'] = df['LoanTermGroups'].apply(lambda x: new_loan_term(x))
    
    return df

In [13]:
def clean3(df, model, cols):
    #convert loan amount to log
    df['LoanAmountLog'] = np.log(df['LoanAmount'].astype('float64'))
    
    #create dummy vars for loan term group
    df['LTG_<15'] = df['LoanTermGroups'].apply(lambda x: 1 if x == '<15' else 0)
    df['LTG_15'] = df['LoanTermGroups'].apply(lambda x: 1 if x == '15' else 0)
    df['LTG_1530'] = df['LoanTermGroups'].apply(lambda x: 1 if x == '(15, 30)' else 0)
    df['LTG_30'] = df['LoanTermGroups'].apply(lambda x: 1 if x == '30' else 0)
    df['LTG_>30'] = df['LoanTermGroups'].apply(lambda x: 1 if x == '>30' else 0)
    
    #predict using model
    df['LA_Predict'] = df.apply(lambda x: predict_func(model, x, cols), axis=1)
    
    #replace missing values with predicted values
    df['LoanAmountLog'].fillna(df['LA_Predict'], inplace=True)
    
    #drop original loan amount column and convert logged back to normal
    df.drop(columns=['LoanAmount', 'LA_Predict'], inplace=True)
    df['LoanAmount'] = np.exp(df['LoanAmountLog'])
    
    #create debt/equity variables
    df['Debt_Equity'] = (df['LoanAmount'] * 1000) / df['FamilyIncome']
    df['Debt_Equity_Annual'] = (((df['LoanAmount']*1000) / df['Loan_Amount_Term']) * 12) / df['FamilyIncome']
    
    return df

In [14]:
def clean4(df, model, cols):
    #predict using model
    df['SE_Predict'] = df.apply(lambda x: predict_func(model, x, cols), axis=1)
    
    #replace missing values with predicted values
    df['Self_Employed'].fillna(df['SE_Predict'], inplace=True)
    
    #convert to dummy variable
    df['Self_Employed_IO'] = df['Self_Employed'].apply(lambda x: 1 if x=='Yes' else 0)
    
    #drop original column
    df.drop(columns=['Self_Employed', 'SE_Predict'], inplace = True)
    
    return df

In [15]:
def fillna_CH(row):
    if np.isnan(row['Credit_History']):
        #check for no education and urban
        if row['NoEd_Urban_IO'] == 1:
            #use model predicted value
            return row['CH_Predict']
        else:
            return 1.0
    else:
        return row['Credit_History']

In [16]:
def clean5(df, model, cols):
    #indicator for no education and urban property area
    df['NoEd_Urban_IO'] = df.apply(lambda x: 1 if x['Education_IO'] == 0 and x['PA_Urban'] == 1 else 0, axis=1)
    
    #predict using model
    df['CH_Predict'] = df.apply(lambda x: predict_func(model, x, cols), axis=1)
    
    #replace missing values with correct value
    df['CH'] = df.apply(lambda x: fillna_CH(x), axis=1)
    
    #drop old column
    df.drop(columns = ['Credit_History', 'NoEd_Urban_IO', 'CH_Predict'], inplace=True)
    
    #rename column
    df.rename(columns = {'CH':'Credit_History'}, inplace=True)
    
    return df

In [28]:
def outlier_id(df, orig):
    income_box = boxplot_stats(orig['FamilyIncome'])[0]
    incLowVal = income_box['whislo']
    incHighVal = income_box['whishi']
    df['income_out_io'] = df['FamilyIncome'].apply(lambda x: 1 if (x < incLowVal or x > incHighVal) else 0)
    
    la_box = boxplot_stats(orig['LoanAmount'])[0]
    laLowVal = la_box['whislo']
    laHighVal = la_box['whishi']
    df['la_out_io'] = df['LoanAmount'].apply(lambda x: 1 if (x < laLowVal or x > laHighVal) else 0)
    
    return df

# Clean and Impute

In [17]:
train = data_grab_online(url, train_name)
test = data_grab_online(url, test_name)

In [18]:
train_clean = clean(train)
test_clean = clean(test)

In [19]:
loan_term_model = pickle.load(open(os.path.join(direct, 'Models', loan_term_model_name), 'rb'))

In [20]:
train_clean2 = clean2(train_clean, loan_term_model, loan_term_cols)
test_clean2 = clean2(test_clean, loan_term_model, loan_term_cols)

In [21]:
loan_amount_model = pickle.load(open(os.path.join(direct, 'Models', loan_amount_model_name), 'rb'))

In [22]:
train_clean3 = clean3(train_clean2, loan_amount_model, loan_amount_cols)
test_clean3 = clean3(test_clean2, loan_amount_model, loan_amount_cols)

In [23]:
self_employed_model = pickle.load(open(os.path.join(direct, 'Models', self_employed_model_name), 'rb'))

In [24]:
train_clean4 = clean4(train_clean3, self_employed_model, self_employed_cols)
test_clean4 = clean4(test_clean3, self_employed_model, self_employed_cols)

In [25]:
credit_history_model = pickle.load(open(os.path.join(direct, 'Models', credit_history_model_name), 'rb'))

In [26]:
train_clean5 = clean5(train_clean4, credit_history_model, credit_history_cols)
test_clean5 = clean5(test_clean4, credit_history_model, credit_history_cols)

In [29]:
original_train = train_clean5.copy()
train_clean6 = outlier_id(train_clean5, original_train)
test_clean6 = outlier_id(test_clean5, original_train)

# Saved clean files for modeling

In [31]:
direct = os.getcwd()
print(direct)

C:\Users\joepo\Desktop\Project Portfolio\Loan Prediction


In [32]:
train_clean6.to_csv(os.path.join(direct, 'forModel_train.csv'), index=False)
test_clean6.to_csv(os.path.join(direct, 'forModel_test.csv'), index=False)